In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming= True,
    callbacks= StreamingStdOutCallbackHandler()
)




In [10]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


In [11]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
        ("human", "{question}")
    ]
)

In [12]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question" : "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [13]:
chef_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are chef"),
        ("human", "I want {cuisine} food recipe")
    ]
)

veg_chef_template = ChatPromptTemplate.from_messages(
    [
        ("system", "youa are vegetarian chef"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_template | chat
veg_chain = veg_chef_template | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

AIMessage(content="I see that you provided a recipe for Chicken Tikka Masala, which is a popular Indian dish. However, as a vegetarian chef, I don't cook with chicken. Would you like me to provide you with a vegetarian version of this dish or suggest a different vegetarian recipe instead? Let me know how I can assist you further!")

In [14]:
program_template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 시를 쓰는 시인이야. 특히 프로그래밍 언어에 대해 잘 알고 있고 그것을 가지고 시를 아주 잘 써"),
        ("human", "{program_lang}으로 프로그래밍 언어에 관한 시를 써줘")
    ]
)

talk_template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 시를 받으면 그 시에 대해 화자의 심정, 비유법 등에 대해 설명하는 선생님이야."),
        ("human", "{poem}에 대해 설명해줘")
    ]
)

program_chain = program_template | chat
talk_chain = talk_template | chat

speaker_chain = {"poem" : program_chain} | talk_chain

speaker_chain.invoke({
    "program_lang" : "java"
})

AIMessage(content='이 시는 자바 프로그래밍에 대한 열정과 매력을 담고 있는 것 같네요. 시인은 한 줄 한 줄 코딩을 펼치며 자바의 세계에 빠져든다고 표현하면서, 클래스와 객체가 춤을 추며 무한한 가능성을 보여준다고 표현하고 있어요. 이는 자바 프로그래밍에서 객체지향적인 접근법과 뛰어난 확장성을 감탄하는 것으로 해석될 수 있겠네요.\n\n또한, 컴파일러의 소리와 실행 결과가 화면에 비춰지는 장면을 통해 프로그래밍의 과정을 생생하게 묘사하고 있습니다. 에러와 예외를 만나도 결국은 해결책을 찾아낸다는 부분은 프로그래밍 과정에서 마주치는 어려움을 극복하며 성장하는 과정을 담은 것 같아요.\n\n마지막으로, 시인은 자바의 매력에 빠져 계속해서 공부하고 싶다고 표현하며, 자신의 작은 작품을 만들어내는 즐거움을 느낀다고 말하고 있습니다. 이는 자바 프로그래밍을 통해 창의적인 작품을 만들어내는 즐거움과 성취감을 경험하는 것을 의미할 수 있겠네요. 전체적으로 이 시는 자바 프로그래밍에 대한 열정과 즐거움을 담고 있는 멋진 작품이라고 할 수 있겠네요.')